In [47]:
# Importeren van packages
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from datetime import datetime, date as dt
from datetime import timedelta
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import scipy
from scipy import stats
import seaborn as sns
import operator
from functools import reduce

In [10]:
# Importeren COOIS and ZSC04
df_1 = pd.read_excel('COOIS.xlsx')
df_2 = pd.read_excel('zcs04.xlsx')
df_3 = pd.read_excel('zcs04.xlsx', sheet_name = 1)

Data from ZCS04

In [122]:
# ZCS04 only use usable columns

df = df_2[df_2.columns[pd.Series(df_2.columns).str.contains("Work center|Target cycle time")]]
# Add material number to left
df.insert(0,"Material" ,df_2['Material'])
# Add mould to left
df.insert(1,"Mould" ,df_2['Mould'])
# Add  insert to left
df.insert(2,"Insert" ,df_2['Insert (mould)'])

# ZCS04 filter out all NaN records
#df = df[df['Work center'].notna()]
#df = df[df['Target cycle time'].notna()]
# Create dataframe with only the cycle times
df_cycle = df[df.columns[pd.Series(df.columns).str.contains("Target cycle time")]]
list_cycle = df_cycle.values.tolist()
df_cycle['mean'] =  df_cycle[df_cycle > .01].mean(axis=1)
df.insert(3, "Mean zcs", df_cycle['mean'])

# Create dataframe with only the work centers
df_wctr = df[df.columns[pd.Series(df.columns).str.contains("Work center")]]

C:\Users\Mark\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [123]:
list_wctr = df_wctr.values.tolist()
list_wctr3 = [[int(i) for i in j if i == i and (str(i).isdigit() or isinstance(i, float))] for j in list_wctr]

In [124]:
df_final = df[['Material','Mould','Insert','Mean zcs']]

In [125]:
# Add workcenters times per material
df_final['Work centers'] = pd.Series(list_wctr3).apply(lambda x: [i for i in x if str(i) != "nan" and str(i).isnumeric()])

C:\Users\Mark\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [126]:
# Add cycle times per material
df_final['Cycle times'] = pd.Series(list_cycle).apply(lambda x: [i for i in x if str(i) != "nan" and str(i) != "0.0"])

C:\Users\Mark\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [140]:
display(df_final)
#df_final.to_excel('zcs04_processed.xlsx')

,Material,Mould,Insert,Mean zcs,Work centers,Cycle times
0,041000007101864807,NaN,NaN,NaN,[],[]
1,078021400002,NaN,NaN,NaN,[],[]
2,1000110004334,W861902-00,I861907-00,45.000000,[350],[45.0]
3,10001100074AW,W861902-00,I861901-00,45.000000,[350],[45.0]
4,100011001601C,W861902-00,I861907-00,45.000000,[350],[45.0]
...,...,...,...,...,...,...
6834,815991020000020,W159900-04,NaN,35.000000,"[351, 354, 356, 373, 374, 366, 369]","[35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0]"
6835,815991020000025,W159900-02,NaN,26.000000,"[333, 345]","[26.0, 26.0]"
6836,815991600000020,W159900-04,NaN,34.444444,"[351, 363, 354, 365, 356, 373, 374, 366, 369]","[35.0, 30.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35...."
6837,815998250000025,W159900-02,NaN,26.000000,"[345, 333, 354]","[26.0, 26.0, 26.0]"


In [128]:
work_centers = (np.unique(reduce(operator.add,list_wctr3)))
print(len(work_centers))
print(work_centers)

59
[301 318 325 326 327 333 334 336 345 346 349 350 351 352 353 354 355 356
 357 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378
 379 402 405 406 407 408 411 412 415 421 422 423 425 426 427 428 429 450
 451 452 453 454 455]


In [129]:
count = 0
for listel in list_wctr3:
    count += len(listel)                    
print('Total Number of elements : ', count)

Total Number of elements :  15425


Data from COOIS

In [130]:
# Create dataframe with average cycle time for every unique product
df_cycle = df_1[['Material Number','Material description','actual cycle SAP']]
df_cycle_m = df_cycle.groupby(['Material Number','Material description']).mean()
material = df_1['Material Number'].unique()
# print(df_cycle_m)

In [131]:
work_center = df_1['Work center'].unique()
# Remove workstations (non-machines)
df_machine = []
amount = 0
for i in work_center:
    if i.isnumeric():
        if 300 < int(i) < 499:
            df_machine.append(i)
            amount += 1
# Print all unique machines
# print(df_machine)
# Print number of machines
# print(amount)

# Remove old machines
df_a = df_1[df_1['Work center'].isin(df_machine)]
df_machine_a = df_a.sort_values('Explosion date').drop_duplicates('Work center',keep='last')
df_machine_b = df_machine_a[['Work center', 'Explosion date']]
df_machine_b['Year'] = df_machine_b['Explosion date'].dt.year
#with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    #display(df_machine_b)
amount1 = 0
work_list = []
for i,row in df_machine_b.iterrows():
    if row['Year'] == 2020:
        work_list.append(row['Work center'])
        amount1 += 1
# print(work_list)
# print(amount1)

C:\Users\Mark\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [139]:
# Calculate average predicted time vs. the actual time spend
# Get average predicted time
df_pred_vs_actual = df_1[df_1['Work center'].isin(df_machine)]
df_pred_vs_actual = df_pred_vs_actual[['Order','Material Number','Explosion date','Delivered quantity','actual cycle SAP','Mould','Work center','Scheduled start time','Scheduled fin. time','ActualStartTime','Actual finish']]
# Calculate total cycle duration
df_pred_vs_actual['Total cycle duration'] = df_pred_vs_actual['Delivered quantity'] * df_pred_vs_actual['actual cycle SAP']
df_pred_vs_actual['Total cycle duration'] =  pd.TimedeltaIndex(df_pred_vs_actual['Total cycle duration'], unit='s')
# Datetime format
df_pred_vs_actual['Explosion year'] = df_pred_vs_actual['Explosion date'].dt.year
t1 = pd.to_datetime(df_pred_vs_actual['Scheduled fin. time'], format = '%H:%M:%S', errors='coerce')
t2 = pd.to_datetime(df_pred_vs_actual['Scheduled start time'], format = '%H:%M:%S', errors='coerce')
t3 = pd.to_datetime(df_pred_vs_actual['ActualStartTime'], format = '%H:%M:%S', errors='coerce')
t4 = pd.to_datetime(df_pred_vs_actual['Actual finish'], format = '%H:%M:%S', errors='coerce')
# Calculate difference of the hours as an absolute value
df_pred_vs_actual['Predicted Total'] = abs(t1 - t2)
df_pred_vs_actual['Actual finish'] = t3 + df_pred_vs_actual['Total cycle duration'] 
df_pred_vs_actual['Actual finish'] = pd.to_datetime(df_pred_vs_actual['Actual finish'], format = '%H:%M:%S')
df_pred_vs_actual['Actual finish'] = df_pred_vs_actual['Actual finish'].dt.time
display(df_pred_vs_actual)
df_pred_vs_actual.to_excel("general_data.xlsx")



,Order,Material Number,Explosion date,Delivered quantity,actual cycle SAP,Mould,Work center,Scheduled start time,Scheduled fin. time,ActualStartTime,Actual finish,Total cycle duration,Explosion year,Predicted Total
0,1245350,702121600000010,2020-06-14,2700,14.244,W021203-20,415,09:00:20,21:30:20,01:37:36,12:18:34.800000,10:40:58.800000,2020,12:30:00
1,1244691,1027300100000,2020-06-21,1260,25.529,W027303-00,455,13:51:06,00:03:06,01:43:10,10:39:16.540000,08:56:06.540000,2020,13:48:00
2,1240249,1053200100038,2020-05-26,2672,16.299,W053203-00,370,11:11:00,23:52:07,02:32:01,14:37:51.928000,12:05:50.928000,2020,12:41:07
3,1243669,706816380003910,2020-06-01,17280,3.772,W368103-00,374,00:13:34,03:00:42,04:54:04,23:00:24.160000,18:06:20.160000,2020,02:47:08
4,1243828,7865010001200JA,2020-04-29,1440,22.000,W859502-10,365,15:22:20,03:18:35,06:54:10,15:42:10,08:48:00,2020,12:03:45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81123,1100021,709426150000020,2012-09-09,2700,24.671,W094203-01,408,11:18:18,14:02:30,17:14:16,11:44:27.700000,18:30:11.700000,2012,02:44:12
81124,1100022,1078560800000,2012-09-08,1440,17.900,W078503-00,411,10:14:42,17:54:18,17:25:48,00:35:24,07:09:36,2012,07:39:36
81125,1100005,1012600100000,2012-09-06,7230,8.179,W012603-10,402,11:12:48,17:12:52,21:13:14,13:38:48.170000,16:25:34.170000,2012,06:00:04
81126,1100167,1088212000005,2012-09-02,396,35.303,W088203-00,424,23:16:28,03:39:27,22:45:49,02:38:48.988000,03:52:59.988000,2012,19:37:01


In [133]:
df_comp = df_pred_vs_actual[['Work center','Material Number','Mould']]
df_comp = df_comp.groupby('Material Number').agg(['unique']).reset_index()

df_comp.to_excel("compatibilityv2.xlsx")

In [134]:
display(df_comp)

,Material Number,Work center,Mould
,,unique,unique
0,1000110004334,[350],[W861902-00]
1,10001100074AW,[350],[W861902-00]
2,10001100177CN,[350],[W861902-00]
3,10001100178CN,[350],[W861902-00]
4,1000110019734,[350],[W861902-00]
...,...,...,...
7790,806210010000044,"[345, 346, 337, 333, 336]",[W062103-10]
7791,815991020000020,[348],[W159900-04]
7792,815991020000025,[345],[W159900-02]


Merge ZCS04 and COOIS in df_final

In [150]:
df_final2 = df_final.merge(df_comp, left_on = 'Material', right_on = 'Material Number', how = 'left')



C:\Users\Mark\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [155]:
df_final3 = df_final2.merge(df_cycle_m, left_on='Material', right_on='Material Number', how = 'left')

In [156]:
display(df_final3)

,Material,Mould,Insert,Mean zcs,Work centers,Cycle times,"(Material Number, )","(Work center, unique)","(Mould, unique)",actual cycle SAP
0,041000007101864807,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
1,078021400002,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN
2,1000110004334,W861902-00,I861907-00,45.000000,[350],[45.0],1000110004334,[350],[W861902-00],48.723562
3,10001100074AW,W861902-00,I861901-00,45.000000,[350],[45.0],10001100074AW,[350],[W861902-00],48.352667
4,100011001601C,W861902-00,I861907-00,45.000000,[350],[45.0],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6834,815991020000020,W159900-04,NaN,35.000000,"[351, 354, 356, 373, 374, 366, 369]","[35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0]",815991020000020,[348],[W159900-04],35.862000
6835,815991020000025,W159900-02,NaN,26.000000,"[333, 345]","[26.0, 26.0]",815991020000025,[345],[W159900-02],26.446000
6836,815991600000020,W159900-04,NaN,34.444444,"[351, 363, 354, 365, 356, 373, 374, 366, 369]","[35.0, 30.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35....",815991600000020,"[351, 373, 365, 358, 348]",[W159900-04],35.467641
6837,815998250000025,W159900-02,NaN,26.000000,"[345, 333, 354]","[26.0, 26.0, 26.0]",815998250000025,"[345, 333, 354, 342]","[W159900-02, W159900-03]",26.315750


In [159]:
df_final3.to_excel('material_info.xlsx')

In [153]:
display(df_final2)

,Material,Mould,Insert,Mean zcs,Work centers,Cycle times,"(Material Number, )","(Work center, unique)","(Mould, unique)"
0,041000007101864807,NaN,NaN,NaN,[],[],NaN,NaN,NaN
1,078021400002,NaN,NaN,NaN,[],[],NaN,NaN,NaN
2,1000110004334,W861902-00,I861907-00,45.000000,[350],[45.0],1000110004334,[350],[W861902-00]
3,10001100074AW,W861902-00,I861901-00,45.000000,[350],[45.0],10001100074AW,[350],[W861902-00]
4,100011001601C,W861902-00,I861907-00,45.000000,[350],[45.0],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6834,815991020000020,W159900-04,NaN,35.000000,"[351, 354, 356, 373, 374, 366, 369]","[35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0]",815991020000020,[348],[W159900-04]
6835,815991020000025,W159900-02,NaN,26.000000,"[333, 345]","[26.0, 26.0]",815991020000025,[345],[W159900-02]
6836,815991600000020,W159900-04,NaN,34.444444,"[351, 363, 354, 365, 356, 373, 374, 366, 369]","[35.0, 30.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35....",815991600000020,"[351, 373, 365, 358, 348]",[W159900-04]
6837,815998250000025,W159900-02,NaN,26.000000,"[345, 333, 354]","[26.0, 26.0, 26.0]",815998250000025,"[345, 333, 354, 342]","[W159900-02, W159900-03]"


In [154]:
display(df_final)

,Material,Mould,Insert,Mean zcs,Work centers,Cycle times
0,041000007101864807,NaN,NaN,NaN,[],[]
1,078021400002,NaN,NaN,NaN,[],[]
2,1000110004334,W861902-00,I861907-00,45.000000,[350],[45.0]
3,10001100074AW,W861902-00,I861901-00,45.000000,[350],[45.0]
4,100011001601C,W861902-00,I861907-00,45.000000,[350],[45.0]
...,...,...,...,...,...,...
6834,815991020000020,W159900-04,NaN,35.000000,"[351, 354, 356, 373, 374, 366, 369]","[35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0]"
6835,815991020000025,W159900-02,NaN,26.000000,"[333, 345]","[26.0, 26.0]"
6836,815991600000020,W159900-04,NaN,34.444444,"[351, 363, 354, 365, 356, 373, 374, 366, 369]","[35.0, 30.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35...."
6837,815998250000025,W159900-02,NaN,26.000000,"[345, 333, 354]","[26.0, 26.0, 26.0]"
